The purpose of this notebook is to demonstrate how to obtain Ortho, DSM and DTM tiles from the API in cases where the ExtractOrthoImages tile isn't appropriate e.g if more than 135 tiles will be required. The notebook does the following:
- Takes a polygon for an AOI geometry
- Determines the tile coordinates for covering that polygon
- Downloads each of the tiles for those coordinates
- Georeferences each PNG tile into a TIF
- creates a mosaic from the set of tiles

## Step 0: set up

In [1]:
import requests
import os

In [2]:
from vdp_python_tools.get_images import tiles_in_polygon, georeference_raster_tile
from vdp_python_tools.authentication import login
from vdp_python_tools.coverage_api_utilities import create_coverage_dataframe


token = login()

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

## Step 1: 

In [ ]:
import geopandas as gpd

gdf_aoi = gpd.read_file("./data/sample/example_boundary.shp")
aoi_geometry = gdf_aoi.iloc[0].geometry
aoi_geometry

It's always a good idea to check the coverage before downloading, so do that next.

In [ ]:
gdf_coverage = create_coverage_dataframe(wkt=aoi_geometry.wkt)
gdf_coverage.head()

In [ ]:
tile_coords = tiles_in_polygon(aoi_geometry, 17)
tile_coords

In [ ]:
import shutil 

format_type = "png"
api = "GetOrthoImageTile/bluesky-high-europe"
image_type = "ortho-high-area"
output_folder = "./data/tmp/"

rasters_to_merge = []
for x, y, zoom in tile_coords:
    url = f"https://api.vexcelgroup.com/images/{api}/{zoom}/{x}/{y}?token={token}"
    if format_type != "tif":
        url += f"&format={format_type}"
    response = requests.get(url, stream = True)
    if response.reason != "OK":
        raise Exception(f"API failed to get tile from api {api} at x={x} y={y} zoom={zoom} see url below \n {url}")
    filepath = os.path.join(output_folder, f"{image_type}_img_{zoom}_{x}_{y}.{format_type}")
    with open(filepath, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)

    if image_type in ["ortho-urban-area", "ortho-high-area"]: # then the tiles aren't georeferenced...yet
        filepath_translated_tile = georeference_raster_tile(x, y, zoom, filepath)
        print(filepath_translated_tile)
        os.remove(filepath)
        rasters_to_merge.append(filepath_translated_tile)
    else: 
        rasters_to_merge.append(filepath)
    
print(f"...finished downloading data")

In [ ]:
import rasterio
from rasterio.merge import merge
from pyproj import CRS

src_files_to_mosaic = []
for filepath in rasters_to_merge:
    src = rasterio.open(filepath)
    src_files_to_mosaic.append(src)

mosaic, out_trans = merge(src_files_to_mosaic)

crs = CRS.from_epsg(4326)
out_meta = src.meta.copy()

# Update the metadata
out_meta.update({"driver": "GTiff",
"height": mosaic.shape[1],
"width": mosaic.shape[2],
"transform": out_trans,
"crs": crs.to_string(), 
})

filepath_mosaic = f"{output_folder}/mosaic.tif"

with rasterio.open(filepath_mosaic, "w", **out_meta) as dest:
    dest.write(mosaic)
print("...done")